# Milestone 1

## Purpose:
In this notebook, we will attempt to download a data dump containing daily rainfall over NSW, Australia dataset found on [figshare](https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681). This data dump is comprised of different files contained in a 776.4 MB compressed format. This size of the uncompressed data dump is approximately 6.6 GB. 

In [1]:
# Imports
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd
from memory_profiler import memory_usage

In [2]:
%load_ext rpy2.ipython
%load_ext memory_profiler

## Creating the API request

We will use the [requests library](https://docs.python-requests.org/en/master/) to generate an http request call to the [figshare API](https://docs.figshare.com/). Specifically, we will make a call to the `/articles` endpoint to retrieve information about the article of interest including the url we need to download the data.

In [7]:
article_id = 14096681
base_url = 'https://api.figshare.com/v2'
headers = {"Content-Type": "application/json"}
endpoint = f'/articles/{article_id}'
output_directory = "rainfall/"

In [9]:
response = requests.request("GET", base_url+endpoint, headers=headers)
data = json.loads(response.text)
data

{'defined_type_name': 'dataset',
 'embargo_date': None,
 'citation': 'Beuzen, Tomas (2021): Daily rainfall over NSW, Australia. figshare. Dataset. https://doi.org/10.6084/m9.figshare.14096681.v3',
 'url_private_api': 'https://api.figshare.com/v2/account/articles/14096681',
 'embargo_reason': '',
 'references': ['https://www.wcrp-climate.org/wgcm-cmip/wgcm-cmip6',
  'https://pangeo-data.github.io/pangeo-cmip6-cloud/',
  'https://www.longpaddock.qld.gov.au/silo/'],
 'funding_list': [],
 'url_public_api': 'https://api.figshare.com/v2/articles/14096681',
 'id': 14096681,
 'custom_fields': [],
 'size': 814109773,
 'metadata_reason': '',
 'funding': None,
 'figshare_url': 'https://figshare.com/articles/dataset/Daily_rainfall_over_NSW_Australia/14096681',
 'embargo_type': 'file',
 'title': 'Daily rainfall over NSW, Australia',
 'defined_type': 3,
 'embargo_options': [],
 'is_embargoed': False,
 'version': 3,
 'resource_doi': None,
 'url_public_html': 'https://figshare.com/articles/dataset/Dai

The response above contains a `data` json key which is of interest of us. It lists the files corresponding to the article along with their download urls.

In [10]:
files = data["files"]           
files

[{'is_link_only': False,
  'name': 'daily_rainfall_2014.png',
  'supplied_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'computed_md5': 'fd32a2ffde300a31f8d63b1825d47e5e',
  'id': 26579150,
  'download_url': 'https://ndownloader.figshare.com/files/26579150',
  'size': 58863},
 {'is_link_only': False,
  'name': 'environment.yml',
  'supplied_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'computed_md5': '060b2020017eed93a1ee7dd8c65b2f34',
  'id': 26579171,
  'download_url': 'https://ndownloader.figshare.com/files/26579171',
  'size': 192},
 {'is_link_only': False,
  'name': 'README.md',
  'supplied_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'computed_md5': '61858c6cc0e6a6d6663a7e4c75bbd88c',
  'id': 26586554,
  'download_url': 'https://ndownloader.figshare.com/files/26586554',
  'size': 5422},
 {'is_link_only': False,
  'name': 'data.zip',
  'supplied_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'computed_md5': 'b517383f76e77bd03755a63a8ff83ee9',
  'id': 26766812,
  'download_url': 'https://